<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-05-30 17:17:46--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8100 (7.9K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   7.91K  --.-KB/s    in 0s      

2022-05-30 17:17:46 (94.2 MB/s) - ‘utils.py’ saved [8100/8100]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

     |████████████████████████████████| 25.9 MB 1.4 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x3/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
xyz_creator = ModelCreator(HUB_URL, 'xyz-model')
xyz_model = xyz_creator.make_model()
callbacks = xyz_creator.get_callbacks()

### First day of training

In [11]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=5,
                        class_weight=class_weight)

Epoch 1/5
519/519 [==============================] - 2656s 5s/step - loss: 293.6083 - precision: 0.6249 - recall: 0.6217 - auc: 0.5135 - accuracy: 0.6224
Epoch 2/5
519/519 [==============================] - 2583s 5s/step - loss: 155.9653 - precision: 0.7146 - recall: 0.7130 - auc: 0.6189 - accuracy: 0.7134
Epoch 3/5
519/519 [==============================] - 2591s 5s/step - loss: 128.7099 - precision: 0.7526 - recall: 0.7516 - auc: 0.6638 - accuracy: 0.7520
Epoch 4/5
519/519 [==============================] - 2591s 5s/step - loss: 125.7976 - precision: 0.7671 - recall: 0.7660 - auc: 0.6804 - accuracy: 0.7664
Epoch 5/5
519/519 [==============================] - 2595s 5s/step - loss: 102.4759 - precision: 0.7901 - recall: 0.7891 - auc: 0.7136 - accuracy: 0.7893


In [17]:
xyz_model.save_weights('drive/MyDrive/checkpoints/xyz-weights.h5')

### Second day

In [18]:
xyz_model.load_weights('drive/MyDrive/checkpoints/xyz-weights.h5')

In [ ]:
history_2 = xyz_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=10,
                          class_weight=class_weight,
                          initial_epoch=5)